# <center> Kafka Producer for Reddit </center>

In [ ]:
!/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --zookeeper clnode222.clemson.cloudlab.us:2181 --replication-factor 1 --partitions 10 --topic test

In [ ]:
!/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --list --zookeeper clnode222.clemson.cloudlab.us:2181

To delete a topic:

Remove the **#** sign to run the cell below. 

In [ ]:
#!/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --zookeeper localhost:2181 --delete --topic test

**Confluent_Kafka**

Official python library from Confluent (company founded by Kafka original developers)
https://docs.confluent.io/current/clients/confluent-kafka-python/

In [ ]:
from confluent_kafka import Producer
import feedparser
import sys
import time
import logging
import json

broker = 'clnode225.clemson.cloudlab.us:6667'
topic = 'test'

# Producer configuration
# See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
conf = {'bootstrap.servers': broker}

# Create Producer instance
p = Producer(**conf)

# Optional per-message delivery callback (triggered by poll() or flush())
# when a message has been successfully delivered or permanently
# failed delivery (after retries).
def delivery_callback(err, msg):
    if err:
        sys.stderr.write('%% Message failed delivery: %s\n' % err)
    else:
        sys.stderr.write('%% Message key %s delivered to %s [%d]\n' % (msg.key(), msg.topic(), msg.partition()))

        
ts = 5
dt = 0.1

for i in range(ts):
    d = feedparser.parse('https://www.reddit.com/r/all/new/.rss')
    print("")
    print("**** PULLING FROM REDDIT  ****")
    print("During this time step, we pull " +  str(len(d.get('entries'))))
    for entry in d.get('entries'):
        print("****  " + entry.get('author') + ": " + entry.get('updated') + ": " + entry.get('title'))
        try:
            p.produce(topic, json.dumps(entry), key=entry.get('title'), callback=delivery_callback)
        except BufferError as e:
            sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))
        p.poll(0)
    time.sleep(dt)

# Wait until all messages have been delivered
sys.stderr.write('%% Waiting for %d deliveries\n' % len(p))
p.flush()